<a href="https://colab.research.google.com/github/prashantksharma/CC-Bot/blob/master/Akita_U_Tabular_Data_Analysis_Sharma_Biomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import os
os.makedirs('./content/drive', exist_ok=True)
drive.flush_and_unmount()

drive.mount('./content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at ./content/drive


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
!ls

content  sample_data


In [5]:
!ls content/drive/Shareddrives/秋田大学_藤山先生/data

ID不一致データ一覧20240827.xlsx
R4年度横手市【オレンジレジストリ】歩行速度データ20240827-2.xlsx
weather_data_20221001_20221130.csv
yokote_stepinfo_2022年10月_2023年5月_3.csv
yokote_stepinfo_2023年12月_2024年3月_2.csv
yokote_stepinfo_2023年6月_2023年11月_2.csv
yokote_stepinfo_2024年4月_2024年6月_2.csv


In [6]:
root_path = 'content/drive/Shareddrives/秋田大学_藤山先生/'

In [7]:
import pandas as pd
import numpy as np

# Step 1: Calculate average and median walking speed from app data

# Load the app data
app_data = pd.read_csv(f'{root_path}/data/yokote_stepinfo_2022年10月_2023年5月_3.csv', encoding='shift-jis')

# Convert '測定開始日時' to datetime, coercing errors to NaT
app_data['測定開始日時'] = pd.to_datetime(app_data['測定開始日時'], errors='coerce')

# Remove rows where '測定開始日時' couldn't be parsed (resulting in NaT)
app_data = app_data.dropna(subset=['測定開始日時'])

# Convert '歩速(m/min)' to numeric, replacing any non-numeric values with NaN
app_data['歩速(m/min)'] = pd.to_numeric(app_data['歩速(m/min)'], errors='coerce')

# Convert '歩速(m/min)' to m/s
app_data['歩速(m/s)'] = app_data['歩速(m/min)'] / 60

# Group by date and ID, then calculate daily average and median walking speed
daily_stats = app_data.groupby([app_data['測定開始日時'].dt.date, 'ID']).agg({
    '歩速(m/s)': ['mean', 'median']
}).reset_index()

# Flatten column names
daily_stats.columns = ['date', 'ID', 'avg_speed_m_s', 'median_speed_m_s']

# Sort by date and ID
daily_stats = daily_stats.sort_values(['date', 'ID'])

# Display the first few rows
print(daily_stats.head())

# Calculate overall average and median
overall_avg_speed = app_data['歩速(m/s)'].mean()
overall_median_speed = app_data['歩速(m/s)'].median()

print(f"\nOverall average walking speed: {overall_avg_speed:.2f} m/s")
print(f"Overall median walking speed: {overall_median_speed:.2f} m/s")

# Save the daily stats to a CSV file
daily_stats.to_csv(f'{root_path}/output/app_data_daily_walking_speed_stats.csv', index=False)
print("\nDaily walking speed stats saved to 'app_data_daily_walking_speed_stats.csv'")

# Save unique IDs to a CSV file
unique_ids = app_data['ID'].unique()
pd.DataFrame({'ID': unique_ids}).to_csv(f'{root_path}/output/unique_ids.csv', index=False)
print(f"\nUnique IDs saved to 'unique_ids.csv'. Total unique IDs: {len(unique_ids)}")

<ipython-input-7-8de3dafa8b2e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_data['歩速(m/min)'] = pd.to_numeric(app_data['歩速(m/min)'], errors='coerce')
<ipython-input-7-8de3dafa8b2e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_data['歩速(m/s)'] = app_data['歩速(m/min)'] / 60


         date        ID  avg_speed_m_s  median_speed_m_s
0  2022-10-13  40012017       0.580164          0.580164
1  2022-10-13  40013501       0.807273          1.028922
2  2022-10-13  40073334       0.642811          0.778229
3  2022-10-13  40095770       1.056734          0.990137
4  2022-10-13  40111155       0.862297          0.985411

Overall average walking speed: 0.83 m/s
Overall median walking speed: 0.86 m/s

Daily walking speed stats saved to 'app_data_daily_walking_speed_stats.csv'

Unique IDs saved to 'unique_ids.csv'. Total unique IDs: 124


In [8]:
# Step 2: Merge weather and app data per day

# Load the daily_stats from Step 1
daily_stats = pd.read_csv(f'{root_path}/output/app_data_daily_walking_speed_stats.csv')

# Convert 'date' to datetime
daily_stats['date'] = pd.to_datetime(daily_stats['date'])

# Load weather data
weather_data = pd.read_csv(f'{root_path}/data/weather_data_20221001_20221130.csv',
                           encoding='shift-jis',
                           skiprows=3,
                           names=['年月日時', '気温(℃)', '気温(℃)_品質情報', '気温(℃)_均質番号',
                                  '降水量(mm)', '降水量(mm)_品質情報', '降水量(mm)_均質番号',
                                  '降雪(cm)', '降雪(cm)_品質情報', '降雪(cm)_均質番号',
                                  '積雪(cm)', '積雪(cm)_品質情報', '積雪(cm)_均質番号',
                                  '日照時間(時間)', '日照時間(時間)_品質情報', '日照時間(時間)_均質番号',
                                  '風速(m/s)', '風速(m/s)_品質情報', '風向', '風向_品質情報', '風速(m/s)_均質番号',
                                  '雲量(10分比)', '雲量(10分比)_品質情報', '雲量(10分比)_均質番号'])

# Convert '年月日時' to datetime with a more flexible approach
weather_data['年月日時'] = pd.to_datetime(weather_data['年月日時'], errors='coerce')

# Remove any rows where the date conversion failed
weather_data = weather_data.dropna(subset=['年月日時'])

# Convert numeric columns in weather_data
numeric_weather_cols = ['気温(℃)', '降水量(mm)', '降雪(cm)', '積雪(cm)', '日照時間(時間)', '風速(m/s)', '雲量(10分比)']
for col in numeric_weather_cols:
    weather_data[col] = pd.to_numeric(weather_data[col], errors='coerce')

# Group weather data by date and calculate daily averages
daily_weather_data = weather_data.groupby(weather_data['年月日時'].dt.date).agg({
    '気温(℃)': 'mean',
    '降水量(mm)': 'sum',
    '降雪(cm)': 'sum',
    '積雪(cm)': 'max',
    '日照時間(時間)': 'sum',
    '風速(m/s)': 'mean',
    '雲量(10分比)': 'mean'
}).reset_index()

daily_weather_data.columns = ['date', '平均気温(℃)', '降水量合計(mm)', '降雪合計(cm)',
                              '最大積雪(cm)', '日照時間合計(時間)', '平均風速(m/s)', '平均雲量(10分比)']

# Convert 'date' to datetime
daily_weather_data['date'] = pd.to_datetime(daily_weather_data['date'])

# Merge daily app stats and weather data
merged_daily_data = pd.merge(daily_stats, daily_weather_data, on='date', how='inner')

# Display results
print("\nMerged daily data sample:")
print(merged_daily_data.head())
print("\nMerged data column types:")
print(merged_daily_data.dtypes)

# Save results to CSV file
merged_daily_data.to_csv(f'{root_path}/output/app_weather_merged_daily_data.csv', index=False)
print("\nMerged daily data saved to 'app_weather_merged_daily_data.csv'")

# Save unique IDs from the merged data
unique_ids_merged = merged_daily_data['ID'].unique()
pd.DataFrame({'ID': unique_ids_merged}).to_csv(f'{root_path}/output/unique_ids_merged.csv', index=False)
print(f"\nUnique IDs from merged data saved to 'unique_ids_merged.csv'. Total unique IDs: {len(unique_ids_merged)}")

<ipython-input-8-57201da3af57>:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_data['年月日時'] = pd.to_datetime(weather_data['年月日時'], errors='coerce')



Merged daily data sample:
        date        ID  avg_speed_m_s  median_speed_m_s    平均気温(℃)  降水量合計(mm)  \
0 2022-10-13  40012017       0.580164          0.580164  11.941667        0.0   
1 2022-10-13  40013501       0.807273          1.028922  11.941667        0.0   
2 2022-10-13  40073334       0.642811          0.778229  11.941667        0.0   
3 2022-10-13  40095770       1.056734          0.990137  11.941667        0.0   
4 2022-10-13  40111155       0.862297          0.985411  11.941667        0.0   

   降雪合計(cm)  最大積雪(cm)  日照時間合計(時間)  平均風速(m/s)  平均雲量(10分比)  
0       0.0       NaN         2.3      1.675         NaN  
1       0.0       NaN         2.3      1.675         NaN  
2       0.0       NaN         2.3      1.675         NaN  
3       0.0       NaN         2.3      1.675         NaN  
4       0.0       NaN         2.3      1.675         NaN  

Merged data column types:
date                datetime64[ns]
ID                          object
avg_speed_m_s              float64


In [37]:
from datetime import datetime

# Step 3: Merge measured data with merged daily data per user ID

# Load the merged daily data from Step 2
merged_daily_data = pd.read_csv(f'{root_path}/output/app_weather_merged_daily_data.csv')
merged_daily_data['date'] = pd.to_datetime(merged_daily_data['date'])

# Load measured data from Excel file
measured_df = pd.read_excel(f'{root_path}/data/R4年度横手市【オレンジレジストリ】歩行速度データ20240827-2.xlsx',
                            header=0,  # Use the first row as header
                            index_col=0,  # Use the first column as index
                            engine='openpyxl')

# # Remove the first row which contains column descriptions
measured_df = measured_df.iloc[1:]

# Rename 'Unnamed: 1' column as the index (this contains the actual IDs)
# Clean and prepare measured_df
measured_df = measured_df.rename(columns={'Unnamed: 1': 'ID'})
measured_df = measured_df.dropna(subset=['ID'])

# print(measured_df)

# Convert numeric columns in measured_df
numeric_columns = ['通常歩行時間（秒）', '通常歩行距離（m）', '1回の測定値 m/s', '握力　㎏']
for col in numeric_columns:
    measured_df[col] = pd.to_numeric(measured_df[col], errors='coerce')


output_filename = f'measured_df_{datetime.now().strftime("%Y%m%d")}.csv'
measured_df.to_csv(f'{root_path}/output/{output_filename}', index=False)


# Convert IDs to same type in both dataframes
measured_df['ID'] = measured_df['ID'].astype(str)
merged_daily_data['ID'] = merged_daily_data['ID'].astype(str)


# Convert ID in merged_daily_data to string
merged_daily_data['ID'] = merged_daily_data['ID'].astype(str)

# For measured_df
measured_df = measured_df.copy()
# Remove rows with empty IDs
measured_df = measured_df[measured_df['ID'].str.len() > 0]

# 2. Set ID as index for measured_df
measured_df.set_index('ID', inplace=True)

# 3. Perform the merge
final_merged_data = pd.merge(
    merged_daily_data,
    measured_df,
    left_on='ID',
    right_index=True,
    how='inner'
)

# 4. Verify the merge
print(f"Records in merged_daily_data: {len(merged_daily_data)}")
print(f"Unique IDs in merged_daily_data: {merged_daily_data['ID'].nunique()}")
print(f"Records in measured_df: {len(measured_df)}")
print(f"Records in final_merged_data: {len(final_merged_data)}")

# Display results
print("\nFinal merged data sample:")
print(final_merged_data.head())

print("\nFinal merged data info:")
print(final_merged_data.info())

print("\nNumber of unique IDs in final merged data:", final_merged_data['ID'].nunique())

# Calculate the percentage of IDs that have measured data
ids_with_measured_data = final_merged_data[final_merged_data['通常歩行時間（秒）'].notna()]['ID'].nunique()
total_ids = final_merged_data['ID'].nunique()
percentage_with_measured_data = (ids_with_measured_data / total_ids) * 100

print(f"\nPercentage of IDs with measured data: {percentage_with_measured_data:.2f}%")

# Save results to CSV file
output_filename = f'final_merged_data_{datetime.now().strftime("%Y%m%d")}.csv'
final_merged_data.to_csv(f'{root_path}/output/{output_filename}', index=False)
print(f"\nFinal merged data saved to '{output_filename}'")

# Additional analysis: Calculate correlations for numeric columns
numeric_cols = final_merged_data.select_dtypes(include=[np.number]).columns
correlation_matrix = final_merged_data[numeric_cols].corr()

print("\nCorrelation Matrix:")
print(correlation_matrix)

# Save correlation matrix to CSV
corr_filename = f'correlation_matrix_{datetime.now().strftime("%Y%m%d")}.csv'
correlation_matrix.to_csv(f'{root_path}/output/{corr_filename}')
print(f"\nCorrelation matrix saved to '{corr_filename}'")

# Print summary statistics for numeric columns
print("\nSummary Statistics:")
print(final_merged_data[numeric_cols].describe())

# Save summary statistics to CSV
stats_filename = f'summary_statistics_{datetime.now().strftime("%Y%m%d")}.csv'
final_merged_data[numeric_cols].describe().to_csv(f'{root_path}/output/{stats_filename}')
print(f"\nSummary statistics saved to '{stats_filename}'")

Records in merged_daily_data: 991
Unique IDs in merged_daily_data: 53
Records in measured_df: 267
Records in final_merged_data: 933

Final merged data sample:
        date        ID  avg_speed_m_s  median_speed_m_s    平均気温(℃)  降水量合計(mm)  \
0 2022-10-13  40012017       0.580164          0.580164  11.941667        0.0   
1 2022-10-13  40013501       0.807273          1.028922  11.941667        0.0   
2 2022-10-13  40073334       0.642811          0.778229  11.941667        0.0   
3 2022-10-13  40095770       1.056734          0.990137  11.941667        0.0   
4 2022-10-13  40111155       0.862297          0.985411  11.941667        0.0   

   降雪合計(cm)  最大積雪(cm)  日照時間合計(時間)  平均風速(m/s)  ...  通常歩行時間（秒）  通常歩行距離（m）  \
0       0.0       NaN         2.3      1.675  ...        4.1          5   
1       0.0       NaN         2.3      1.675  ...        4.5          5   
2       0.0       NaN         2.3      1.675  ...        3.0          5   
3       0.0       NaN         2.3      1.675  ...     

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


# Additional analysis: Calculate correlations for numeric columns
numeric_cols = final_merged_data.select_dtypes(include=[np.number]).columns
correlation_matrix = final_merged_data[numeric_cols].corr()

# Create a correlation heatmap
plt.figure(figsize=(20, 16))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Heatmap of Numeric Variables')
plt.tight_layout()

# Save the heatmap
heatmap_filename = f'correlation_heatmap_{datetime.now().strftime("%Y%m%d")}.png'
plt.savefig(f'{root_path}/output/{heatmap_filename}', dpi=300, bbox_inches='tight')
print(f"\nCorrelation heatmap saved to '{heatmap_filename}'")

# Close the plot to free up memory
plt.close()

/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 24179 (\N{CJK UNIFIED IDEOGRAPH-5E73}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 22343 (\N{CJK UNIFIED IDEOGRAPH-5747}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 27671 (\N{CJK UNIFIED IDEOGRAPH-6C17}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 28201 (\N{CJK UNIFIED IDEOGRAPH-6E29}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 38477 (\N{CJK UNIFIED IDEOGRAPH-964D}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 27700 (\N{CJK UNIFIED IDEOGRAPH-6C34}) missing from current font.
  fig.canvas.draw()
/usr


Correlation heatmap saved to 'correlation_heatmap_20241025.png'
